
# Hyperparameter tuning with nested cross-validation

In this notebook, we will illustrate how to perform hyperparameter tuning
using nested cross-validation.

We start be defining the predictive model that we created in the previous
notebook.

In [ ]:
import pandas as pd

df = pd.read_csv("../datasets/penguins.csv")
df

In [ ]:
feature_names = [
    "Region",
    "Island",
    "Culmen Depth (mm)",
    "Flipper Length (mm)",
    "Body Mass (g)",
    "Sex",
]
target_name = "Species"
X = df[feature_names]
y = df[target_name]

categorical_columns = X.select_dtypes(include="object").columns
X[categorical_columns] = X[categorical_columns].astype("category")

In [ ]:
from skrub import tabular_learner
from sklearn.linear_model import LogisticRegression

logistic_regression = tabular_learner(LogisticRegression())
logistic_regression


In the past, we saw how to evaluate this model with cross-validation.

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate

cv_results = cross_validate(logistic_regression, X, y, cv=5, return_train_score=True)
cv_results = pd.DataFrame(cv_results)
cv_results[["train_score", "test_score"]]


However, this machine learning pipeline has some hyperparameters that are set to
default values and not necessarily optimal for our problem.

In [ ]:
logistic_regression.get_params()


We can tune these hyperparameters using a search strategy.

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    "simpleimputer__strategy": ["mean", "median", "most_frequent"],
    "logisticregression__C": loguniform(1e-3, 1e3),
}
tuned_model = RandomizedSearchCV(
    logistic_regression,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    random_state=0,
)
tuned_model

In [ ]:
# set `return_estimator=True` to access the best model found during the search
cv_results = cross_validate(tuned_model, X, y, cv=5, return_estimator=True)
cv_results = pd.DataFrame(cv_results)
cv_results


We can check the best estimator that have been for each fold.

In [ ]:
for estimator in cv_results["estimator"]:
    print(estimator.best_params_)


However, the previous approach is not the best way to tune hyperparameters.
We used the default score that is the accuracy here. However, if we have a
probabilistic model, we should instead optimized a proper scoring function instead
of a threshold-based metric. So we can optimize the log loss instead of the accuracy.

In [ ]:
cv_results = cross_validate(
    tuned_model, X, y, cv=5, scoring="neg_log_loss", return_estimator=True
)
cv_results = pd.DataFrame(cv_results)
cv_results

In [ ]:
for estimator in cv_results["estimator"]:
    print(estimator.best_params_)